## Classification:
Classify text into categories or labels using chat models with structured outputs.  
[Link](https://python.langchain.com/docs/tutorials/classification/)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.
    - sentiment (positive, negative, or neutral)
    - aggressiveness (scale of 1-10)
    - language (the language the text is written in)

Please provide the information in a structured format that matches the expected schema.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


llm = ChatGroq(temperature=0, model="llama3-8b-8192").with_structured_output(
    Classification
)

In [11]:
inp = "म अत्यन्त खुसी छु कि मैले तपाईंलाई भेटें! मलाई लाग्छ हामी धेरै राम्रो साथी हुनेछौं!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=5, lang='Nepali')

In [13]:
response.model_dump()

{'sentiment': 'positive', 'aggressiveness': 5, 'lang': 'Nepali'}

### Finer control

Careful schema definition gives us more control over the model's output

In [14]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1,2,3,4,5]
    )
    language: str = Field(
        ..., enum=['spanish', 'english', 'french', 'nepali', 'hindi']
    )

In [19]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.
    - sentiment
    - aggressiveness
    - language

Please provide the information in a structured format that matches the expected schema.

Passage:
{input}
"""
)

llm = ChatGroq(temperature=0, model="llama3-8b-8192").with_structured_output(
    Classification
)

In [20]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish')